<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-09-28 18:55:53--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  11.5MB/s    in 0.5s    

2022-09-28 18:55:54 (11.5 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-09-28 18:55:54--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [ ]:
! pip install -q deepmatch

     |████████████████████████████████| 147 kB 13.1 MB/s 
     |████████████████████████████████| 2.9 MB 64.7 MB/s 


In [168]:
import pandas as pd
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize, reduce_mean

# Data

In [ ]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
print(data.shape)
print(data.user_id.nunique())
print(data.movie_id.nunique())
data.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(10000, 10)
46
2175


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation,F,1,10,48067


# Feature Engineering

In [ ]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [ ]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

In [ ]:
feature_max_idx

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [ ]:
data.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18


In [ ]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')
user_profile.shape, item_profile.shape

((46, 4), (2175, 1))

In [ ]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list.head()

user_id
1    [644, 385, 483, 1881, 1306, 646, 731, 1570, 35...
2    [644, 1735, 1283, 1077, 656, 1121, 692, 773, 1...
3    [644, 1570, 487, 647, 357, 693, 909, 670, 72, ...
4    [644, 1881, 1735, 1283, 320, 1545, 152, 1121, ...
5    [644, 1306, 359, 1283, 405, 715, 320, 604, 154...
Name: movie_id, dtype: object

In [ ]:
train_set, test_set = gen_data_set(data, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 46/46 [00:00<00:00, 179.68it/s]

8 8


In [ ]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [ ]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((19816,), (19816,), (46,), (46,))

In [ ]:
train_model_input

{'user_id': array([21, 42, 43, ..., 15, 29, 37]),
 'movie_id': array([ 807,  203, 1728, ...,  703, 1770, 2072]),
 'hist_movie_id': array([[1736,  961,   75, ...,   10,  286,  287],
        [ 216,  471,   31, ..., 1349,  408, 1522],
        [ 669, 1734, 1094, ..., 1052, 1051,  495],
        ...,
        [ 228,  851, 1338, ..., 2172, 1165,   19],
        [1976, 1842, 2077, ...,    0,    0,    0],
        [ 213, 1358, 1097, ...,  264,  909, 1686]], dtype=int32),
 'hist_genres': array([[ 1,  8,  5, ...,  5,  8,  1],
        [ 5,  8,  5, ...,  1,  1,  1],
        [ 1,  1, 11, ...,  5,  8, 14],
        ...,
        [ 4,  5,  5, ...,  5,  5,  1],
        [ 1,  5,  2, ...,  0,  0,  0],
        [ 1,  1,  1, ...,  5,  5,  5]], dtype=int32),
 'hist_len': array([50, 50, 50, ..., 50, 30, 50]),
 'genres': array([ 5,  8,  8, ..., 11,  5,  1]),
 'gender': array([1, 2, 2, ..., 2, 2, 1]),
 'age': array([4, 2, 3, ..., 3, 4, 3]),
 'occupation': array([ 4,  5,  6, ...,  5, 16, 17]),
 'zip': array([44, 37, 

# Specify Model Features Input

In [ ]:
train_model_input

{'user_id': array([21, 42, 43, ..., 15, 29, 37]),
 'movie_id': array([ 807,  203, 1728, ...,  703, 1770, 2072]),
 'hist_movie_id': array([[1736,  961,   75, ...,   10,  286,  287],
        [ 216,  471,   31, ..., 1349,  408, 1522],
        [ 669, 1734, 1094, ..., 1052, 1051,  495],
        ...,
        [ 228,  851, 1338, ..., 2172, 1165,   19],
        [1976, 1842, 2077, ...,    0,    0,    0],
        [ 213, 1358, 1097, ...,  264,  909, 1686]], dtype=int32),
 'hist_genres': array([[ 1,  8,  5, ...,  5,  8,  1],
        [ 5,  8,  5, ...,  1,  1,  1],
        [ 1,  1, 11, ...,  5,  8, 14],
        ...,
        [ 4,  5,  5, ...,  5,  5,  1],
        [ 1,  5,  2, ...,  0,  0,  0],
        [ 1,  1,  1, ...,  5,  5,  5]], dtype=int32),
 'hist_len': array([50, 50, 50, ..., 50, 30, 50]),
 'genres': array([ 5,  8,  8, ..., 11,  5,  1]),
 'gender': array([1, 2, 2, ..., 2, 2, 1]),
 'age': array([4, 2, 3, ..., 3, 4, 3]),
 'occupation': array([ 4,  5,  6, ...,  5, 16, 17]),
 'zip': array([44, 37, 

In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['movie_id'])]
len(train_counter), type(train_counter), len(item_count)

(2175, collections.Counter, 2176)

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [ ]:
# item features
item_feature_columns = ['movie_id']
# User features
user_feature_columns = ['user_id',
                        "gender",
                        "age",
                        "occupation",
                        "zip",
                        'hist_movie_id',
                        'hist_genres',
                        'hist_len'
                        ]


In [ ]:
feature_max_idx

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [ ]:
user_sparse_feature_columns = user_feature_columns[0:5]
user_sparse_feature_columns

['user_id', 'gender', 'age', 'occupation', 'zip']

In [ ]:
user_seq_sparse_feature_columns = user_feature_columns[5:-1]
user_seq_sparse_feature_columns 

['hist_movie_id', 'hist_genres']

In [ ]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=True)
   emb.trainable = True
   sparse_embedding[name] = emb

In [ ]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [ ]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6467c190>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466ec90>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466bb90>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466bbd0>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466b190>,
 'hist_movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa64672410>,
 'hist_genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466b810>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa64687c50>}

In [ ]:
sparse_embedding

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6467c190>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466ec90>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466bb90>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466bbd0>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466b190>,
 'hist_movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa64672410>,
 'hist_genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa6466b810>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ffa64687c50>}

In [ ]:
# youtubeDNN
input_from_feature_columns(user_features, user_feature_columns,l2_reg_embedding, seed=seed, embedding_matrix_dict=embedding_matrix_dict)

input_from_feature_columns(features, feature_columns, l2_reg, seed, prefix='', seq_mask_zero=True,
                               support_dense=True, support_group=False):
  

In [ ]:
# def get_varlen_pooling_list(embedding_dict, features, varlen_sparse_feature_columns, to_list=False):
#     pooling_vec_list = defaultdict(list)
#     for fc in varlen_sparse_feature_columns:
#         feature_name = fc.name
#         combiner = fc.combiner
#         feature_length_name = fc.length_name
#         if feature_length_name is not None:
#             # if fc.weight_name is not None:
#             #     seq_input = WeightedSequenceLayer(weight_normalization=fc.weight_norm)(
#             #         [embedding_dict[feature_name], features[feature_length_name], features[fc.weight_name]])
#             # else:
#                 seq_input = embedding_dict[feature_name]
#             vec = SequencePoolingLayer(combiner, supports_masking=False)(
#                 [seq_input, features[feature_length_name]])
#         # else:
#         #     if fc.weight_name is not None:
#         #         seq_input = WeightedSequenceLayer(weight_normalization=fc.weight_norm, supports_masking=True)(
#         #             [embedding_dict[feature_name], features[fc.weight_name]])
#         #     else:
#         #         seq_input = embedding_dict[feature_name]
#         #     vec = SequencePoolingLayer(combiner, supports_masking=True)(
#         #         seq_input)
#         pooling_vec_list[fc.group_name].append(vec)
#     if to_list:
#         return chain.from_iterable(pooling_vec_list.values())
#     return pooling_vec_list

In [ ]:


# from itertools import chain

# from deepctr.feature_column import SparseFeat, VarLenSparseFeat, create_embedding_matrix, embedding_lookup, \
#     get_dense_input, varlen_embedding_lookup, get_varlen_pooling_list, mergeDict


# def input_from_feature_columns(features, feature_columns, l2_reg, seed, prefix='', seq_mask_zero=True,
#                                support_dense=True, support_group=False, embedding_matrix_dict=None):
#     # sparse_feature_columns = list(
#     #     filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if feature_columns else []
#     # varlen_sparse_feature_columns = list(
#     #     filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if feature_columns else []
#     if embedding_matrix_dict is None:
#         embedding_matrix_dict = create_embedding_matrix(feature_columns, l2_reg, seed, prefix=prefix,
#                                                         seq_mask_zero=seq_mask_zero)

#     group_sparse_embedding_dict = embedding_lookup(embedding_matrix_dict, features, sparse_feature_columns)
#     # dense_value_list = get_dense_input(features, feature_columns)
#     # if not support_dense and len(dense_value_list) > 0:
#     #     raise ValueError("DenseFeat is not supported in dnn_feature_columns")

#     sequence_embed_dict = varlen_embedding_lookup(embedding_matrix_dict, features, varlen_sparse_feature_columns)
#     group_varlen_sparse_embedding_dict = get_varlen_pooling_list(sequence_embed_dict, features,
#                                                                  varlen_sparse_feature_columns)
#     group_embedding_dict = mergeDict(group_sparse_embedding_dict, group_varlen_sparse_embedding_dict)
#     if not support_group:
#         group_embedding_dict = list(chain.from_iterable(group_embedding_dict.values()))
#     return group_embedding_dict, dense_value_list

### Create Input Tensor

In [ ]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

In [ ]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'gender:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'age:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'occupation:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_movie_id:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_genres:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len:0' shape=(None, 1) dtype=int32>]

In [ ]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [ ]:
item_inputs_list

[<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>]

### Embed Inputs

In [ ]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

In [ ]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

In [ ]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

In [ ]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

In [ ]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_gender/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_age/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_occupation/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_zip/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d_19/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_20/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

### Combine Embeded Input (User only)

In [ ]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [ ]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten_1/Reshape:0' shape=(None, 144) dtype=float32>

## Model Specification

### User Layers

In [ ]:
from tensorflow.keras import layers

In [ ]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [ ]:
# build model
from tensorflow.python.keras.layers import Layer, Dropout, Activation
from tensorflow.python.ops.init_ops_v2 import Zeros, glorot_normal
from tensorflow.keras.layers import BatchNormalization
class DNN(Layer):
    """The Multi Layer Percetron
      Input shape
        - nD tensor with shape: ``(batch_size, ..., input_dim)``. The most common situation would be a 2D input with shape ``(batch_size, input_dim)``.
      Output shape
        - nD tensor with shape: ``(batch_size, ..., hidden_size[-1])``. For instance, for a 2D input with shape ``(batch_size, input_dim)``, the output would have shape ``(batch_size, hidden_size[-1])``.
      Arguments
        - **hidden_units**:list of positive integer, the layer number and units in each layer.
        - **activation**: Activation function to use.
        - **l2_reg**: float between 0 and 1. L2 regularizer strength applied to the kernel weights matrix.
        - **dropout_rate**: float in [0,1). Fraction of the units to dropout.
        - **use_bn**: bool. Whether use BatchNormalization before activation or not.
        - **output_activation**: Activation function to use in the last layer.If ``None``,it will be same as ``activation``.
        - **seed**: A Python integer to use as random seed.
    """

    def __init__(self, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False, output_activation=None,
                 seed=1024, **kwargs):
        self.hidden_units = hidden_units
        self.activation = activation
        self.l2_reg = l2_reg
        self.dropout_rate = dropout_rate
        self.use_bn = use_bn
        self.output_activation = output_activation
        self.seed = seed

        super(DNN, self).__init__(**kwargs)

    def build(self, input_shape):
        # if len(self.hidden_units) == 0:
        #     raise ValueError("hidden_units is empty")
        input_size = input_shape[-1]
        hidden_units = [int(input_size)] + list(self.hidden_units)
        self.kernels = [self.add_weight(name='kernel' + str(i),
                                        shape=(
                                            hidden_units[i], hidden_units[i + 1]),
                                        initializer=glorot_normal(
                                            seed=self.seed),
                                        regularizer=l2(self.l2_reg),
                                        trainable=True) for i in range(len(self.hidden_units))]
        self.bias = [self.add_weight(name='bias' + str(i),
                                     shape=(self.hidden_units[i],),
                                     initializer=Zeros(),
                                     trainable=True) for i in range(len(self.hidden_units))]
        if self.use_bn:
            self.bn_layers = [BatchNormalization() for _ in range(len(self.hidden_units))]

        self.dropout_layers = [Dropout(self.dropout_rate, seed=self.seed + i) for i in
                               range(len(self.hidden_units))]

        self.activation_layers = [Activation(self.activation) for _ in range(len(self.hidden_units))]

        if self.output_activation:
            self.activation_layers[-1] = Activation(self.output_activation)

        super(DNN, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, training=None, **kwargs):

        deep_input = inputs

        for i in range(len(self.hidden_units)):
            fc = tf.nn.bias_add(tf.tensordot(
                deep_input, self.kernels[i], axes=(-1, 0)), self.bias[i])

            if self.use_bn:
                fc = self.bn_layers[i](fc, training=training)
            try:
                fc = self.activation_layers[i](fc, training=training)
            except TypeError as e:  # TypeError: call() got an unexpected keyword argument 'training'
                print("make sure the activation function use training flag properly", e)
                fc = self.activation_layers[i](fc)

            fc = self.dropout_layers[i](fc, training=training)
            deep_input = fc

        return deep_input

    def compute_output_shape(self, input_shape):
        if len(self.hidden_units) > 0:
            shape = input_shape[:-1] + (self.hidden_units[-1],)
        else:
            shape = input_shape

        return tuple(shape)

    def get_config(self, ):
        config = {'activation': self.activation, 'hidden_units': self.hidden_units,
                  'l2_reg': self.l2_reg, 'use_bn': self.use_bn, 'dropout_rate': self.dropout_rate,
                  'output_activation': self.output_activation, 'seed': self.seed}
        base_config = super(DNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def l2_normalize(x, axis=-1):
    return Lambda(lambda x: tf.nn.l2_normalize(x, axis))(x)

In [ ]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



In [ ]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda_2/l2_normalize:0' shape=(None, 32) dtype=float32>

### Item Layer

In [ ]:
class EmbeddingIndex(Layer):

    def __init__(self, index, **kwargs):
        self.index = index
        super(EmbeddingIndex, self).__init__(**kwargs)

    def build(self, input_shape):
        super(EmbeddingIndex, self).build(
            input_shape)  # Be sure to call this somewhere!

    def call(self, x, **kwargs):
        return tf.constant(self.index)

    def get_config(self, ):
        config = {'index': self.index, }
        base_config = super(EmbeddingIndex, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [172]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

#  embed the item index input

# item_embedding_weight = NoMask()(sparse_embedding[item_feature_name])
# pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
item_embeded_index = sparse_embedding[item_feature_name](item_index)
item_embeded_index = l2_normalize(item_embeded_input)
item_embeded_index


<tf.Tensor 'lambda_4/l2_normalize:0' shape=(2176, 32) dtype=float32>

In [173]:
item_embeded_index.eval(session=K.get_session()).shape

(2176, 32)

In [ ]:
# .eval(session=K.get_session())

### Output

In [ ]:
from tensorflow.python.keras.models import Model

In [ ]:

output = SampledSoftmaxLayer(sampler_config._asdict(), temperature)(
    [item_embeded_index, user_dnn_out, item_features[item_feature_name]])

In [175]:
user_dnn_output

<tf.Tensor 'lambda_2/l2_normalize:0' shape=(None, 32) dtype=float32>